You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:

* The Times Higher Education World University Ranking, shortly *Times*,
* The Academic Ranking of World Universities, shortly *Shanghai*,
* The Center for World University Rankings, shortly *cwur*.

Notes

* It is mandatory to use GitHub for developing the project.
* The project must be a jupyter notebook.
* There is no restriction on the libraries that can be used, nor on the Python version.
* All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

# PROJECT 2020-2021 - Gruppo 37 - Foundations of Computer Science
### Professor Gianluca Della Vedova

##### Autori: Marco Braga, Alessandro Maccario

Prima di tutto vengono importate le librerie necessarie nel proseguo dell'analisi.

In [1]:
import re
import math
import pprint
import numpy as np
import pandas as pd

### 1. For each university, extract from the `times` dataset the most recent and the least recent data, obtaining two separate dataframes

Viene caricato il dataset `times` e ne viene visualizzato il contenuto:

In [31]:
times = pd.read_csv('dataset_progetto/timesData.csv')
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


Viene creato il dataset `times_min` per immagazzinare i dati meno recenti:
(viene usata la funzione `loc` in quanto si accede ai dati tramite una *label* specifica) 

In [32]:
times_min = times.loc[times.groupby('university_name')['year'].idxmin()]
times_min

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,"2,400",7.9,20%,46 : 54,2011
99,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,"2,218",8.0,14%,49 : 51,2011
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,-,69.5,"2,429",4.8,30%,18 : 82,2011
47,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,"9,666",10.5,54%,27 : 73,2011


Viene creato il dataset `times_max` per immagazzinare i dati più recenti:

In [33]:
# new dataset with most recent data for each university
times_max = times.loc[times.groupby('university_name')['year'].idxmax()]
times_max

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14%,49 : 51,2016
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54%,27 : 73,2016


### 2. For each university, compute the improvement in income between the least recent and the most recent data points

La colonna `income` presenta molti valori `-` che verranno considerati come valori nulli o mancanti. In questo caso si è deciso di escluderli dal computo delle differenze per considerare solamente quelle università che presentavano sia nel dataset dei valori minimi che nel dataset dei valori massimo valori non mancanti, per permettere il calcolo della differenza effettiva.

Si crea un dataset unico che contenga quindi i valori di `times_min` e di `times_max`.

In [34]:
times_income = pd.merge(times_min, times_max, on='university_name', suffixes=('_min', '_max'))
times_income.head()

,world_rank_min,university_name,country_min,teaching_min,international_min,research_min,citations_min,income_min,total_score_min,num_students_min,...,international_max,research_max,citations_max,income_max,total_score_max,num_students_max,student_staff_ratio_max,international_students_max,female_male_ratio_max,year_max
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",...,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",...,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",...,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",...,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",...,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


Da quest'ultimo si estraggono solo quelle colonne di interesse:

In [35]:
times_income = times_income[['university_name', 'income_min', 'income_max']]
times_income.head()

,university_name,income_min,income_max
0,AGH University of Science and Technology,-,-
1,Aalborg University,36.4,43.7
2,Aalto University,61.9,61.6
3,Aarhus University,61.5,68.3
4,Aberystwyth University,35.5,31.3


Si decide di prendere quindi in considerazione solamente gli elementi non contenenti il dash (`-`) per poter poi calcolare le differenze dei valori.

In [36]:
times_income = times_income[(~times_income['income_min'].str.contains('-')) & (~times_income['income_max'].str.contains('-'))]
times_income

,university_name,income_min,income_max
1,Aalborg University,36.4,43.7
2,Aalto University,61.9,61.6
3,Aarhus University,61.5,68.3
4,Aberystwyth University,35.5,31.3
5,Adam Mickiewicz University,28.7,28.7
...,...,...,...
812,Zhejiang University,70.3,96.2
813,École Normale Supérieure,30.7,37.1
814,École Normale Supérieure de Lyon,26.1,31.7
816,École Polytechnique Fédérale de Lausanne,38.0,65.4


Si trasformano in float i valori di colonna:

In [37]:
times_income['income_min'] = times_income['income_min'].astype('float')
times_income['income_max'] = times_income['income_max'].astype('float')

In [38]:
times_income['income_diff'] = times_income['income_max'] - times_income['income_min']
times_income

,university_name,income_min,income_max,income_diff
1,Aalborg University,36.4,43.7,7.3
2,Aalto University,61.9,61.6,-0.3
3,Aarhus University,61.5,68.3,6.8
4,Aberystwyth University,35.5,31.3,-4.2
5,Adam Mickiewicz University,28.7,28.7,0.0
...,...,...,...,...
812,Zhejiang University,70.3,96.2,25.9
813,École Normale Supérieure,30.7,37.1,6.4
814,École Normale Supérieure de Lyon,26.1,31.7,5.6
816,École Polytechnique Fédérale de Lausanne,38.0,65.4,27.4


Infine, si controlla se sono presenti ancora valori `-` oppure valori nulli: (viene usato il metodo `.isin` poiché ad usare l'operatore di uguaglianza si ha un conflitto fra numpy e pandas)

In [39]:
times_income[times_income['income_diff'].isin(['^-$'])]

,university_name,income_min,income_max,income_diff


In [40]:
times_income[times_income['income_diff'].isna()]

,university_name,income_min,income_max,income_diff


### 3. Find the university with the largest increase computed in the previous point

Valutiamo tramite la funzione `.max()` la presenza di uno o più record che presentano valori massimi di differenza fra gli `income` dalla data più recente a quella meno recente.

In [41]:
times_income[times_income['income_diff'] == times_income['income_diff'].max()]

,university_name,income_min,income_max,income_diff
428,TU Dresden,31.9,99.7,67.8


### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

Consideriamo i dataset utili al nostro scopo, ovvero `cwur.csv`, `shanghaiData` e `times` e isoliamo i più recenti data point e le colonne di nostro interesse, ovvero `world rank`, `university name/institution`, `year`.

In [42]:
shanghai_db = pd.read_csv('dataset_progetto/shanghaiData.csv')
shanghai_db.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005


In [43]:
cwur_db = pd.read_csv('dataset_progetto/cwurData.csv')
cwur_db.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012


In [44]:
times_for_max = times.iloc[times.groupby('university_name')['year'].idxmax()][['university_name','world_rank','year']]
times_for_max.head()

,university_name,world_rank,year
2405,AGH University of Science and Technology,601-800,2016
2003,Aalborg University,201-250,2016
2056,Aalto University,251-300,2016


In [45]:
cwurdb_for_max = cwur_db.iloc[cwur_db.groupby('institution')['year'].idxmax()][['institution','world_rank','year']]
cwurdb_for_max.head()

,institution,world_rank,year
1981,AGH University of Science and Technology,782,2015
1764,Aalborg University,565,2015
1620,Aalto University,421,2015


In [46]:
shanghaidb_for_max = shanghai_db.iloc[shanghai_db.groupby('university_name')['year'].idxmax()][['university_name','world_rank','year']]
shanghaidb_for_max.head()

,university_name,world_rank,year
4697,Aalborg University,301-400,2015
4797,Aalto University,401-500,2015
4469,Aarhus University,73,2015


Volendo ottenere un unico dataset finale, si dovrà fare prima un `merge` fra due dataset e poi un'ulteriore `merge` sui dati rimanenti:

In [47]:
fuso_1 = pd.merge(times_for_max, shanghaidb_for_max, on='university_name', suffixes=('_times', '_shanghai'))
fuso_1.head()

,university_name,world_rank_times,year_times,world_rank_shanghai,year_shanghai
0,Aalborg University,201-250,2016,301-400,2015
1,Aalto University,251-300,2016,401-500,2015
2,Aarhus University,=106,2016,73,2015
3,Aix-Marseille University,251-300,2016,102-150,2011
4,Aristotle University of Thessaloniki,601-800,2016,401-500,2015


In [48]:
times_shanghai_cwur = pd.merge(fuso_1, cwurdb_for_max, left_on='university_name', right_on='institution')
times_shanghai_cwur.head()

,university_name,world_rank_times,year_times,world_rank_shanghai,year_shanghai,institution,world_rank,year
0,Aalborg University,201-250,2016,301-400,2015,Aalborg University,565,2015
1,Aalto University,251-300,2016,401-500,2015,Aalto University,421,2015
2,Aarhus University,=106,2016,73,2015,Aarhus University,122,2015
3,Aix-Marseille University,251-300,2016,102-150,2011,Aix-Marseille University,206,2015
4,Aristotle University of Thessaloniki,601-800,2016,401-500,2015,Aristotle University of Thessaloniki,459,2015


Poiché nella colonna `world_rank_times` sono presenti elementi con il simbolo `=` al loro interno, è necessario rimpiazzare tale elemento per poter lavorare agevolmente con i valori che seguono il simbolo.

In [49]:
times_shanghai_cwur['world_rank_times'] = times_shanghai_cwur['world_rank_times'].str.replace('=', '')

Controlliamo che tutti i valori siano stati eliminati.

In [50]:
times_shanghai_cwur[times_shanghai_cwur['world_rank_times'].str.contains('=')]

,university_name,world_rank_times,year_times,world_rank_shanghai,year_shanghai,institution,world_rank,year


Controlliamo inoltre che non siano presenti valori nulli o celle che presentano il solo valore `-`:

In [51]:
times_shanghai_cwur[times_shanghai_cwur['world_rank_times'].isna()]

,university_name,world_rank_times,year_times,world_rank_shanghai,year_shanghai,institution,world_rank,year


In [52]:
times_shanghai_cwur[times_shanghai_cwur['world_rank_times'].str.contains('^-$')]

,university_name,world_rank_times,year_times,world_rank_shanghai,year_shanghai,institution,world_rank,year


Nel passaggio successivo bisogna affrontare le differenze delle diverse colonne di world rank: in almeno due sono presenti dei range di valori che non permettono nell'immediato di poter ottenere delle differenze di valori. Prima si dovranno trattare adeguatamente e, solo successivamente, potranno essere considerati come valori numerici.

In [53]:
def differenza_max_min(elemento):
    # Match con gli elementi che contengono la dicitura richiesta
    str_times = re.match('(?P<inf>\d+)-(?P<sup>\d+)', elemento['world_rank_times'])
    str_shanghai = re.match('(?P<inf>\d+)-(?P<sup>\d+)', elemento['world_rank_shanghai'])
    
    # se str_times è True
    if str_times:
        sup_times = int(str_times.group('sup'))
        inf_times = int(str_times.group('inf'))
    else:
        sup_times = int(elemento['world_rank_times'])
        inf_times = int(elemento['world_rank_times'])
    if str_shanghai:
        sup_shanghai = int(str_shanghai.group('sup'))
        inf_shanghai = int(str_shanghai.group('inf'))
    else:
        sup_shanghai = int(elemento['world_rank_shanghai'])
        inf_shanghai = int(elemento['world_rank_shanghai'])
    
    cwur_elem = elemento['world_rank']
    massimo = max(sup_times, sup_shanghai, cwur_elem)
    minimo = min(sup_times, sup_shanghai, cwur_elem)
    
    # Viene ritornata la differenza fra i valori
    return massimo - minimo   

Viene applicata la funzione precedentemente creata:

In [54]:
times_shanghai_cwur['differenza_max_min'] = times_shanghai_cwur.apply(differenza_max_min, axis=1)
times_shanghai_cwur

,university_name,world_rank_times,year_times,world_rank_shanghai,year_shanghai,institution,world_rank,year,differenza_max_min
0,Aalborg University,201-250,2016,301-400,2015,Aalborg University,565,2015,315
1,Aalto University,251-300,2016,401-500,2015,Aalto University,421,2015,200
2,Aarhus University,106,2016,73,2015,Aarhus University,122,2015,49
3,Aix-Marseille University,251-300,2016,102-150,2011,Aix-Marseille University,206,2015,150
4,Aristotle University of Thessaloniki,601-800,2016,401-500,2015,Aristotle University of Thessaloniki,459,2015,341
...,...,...,...,...,...,...,...,...,...
357,Yale University,12,2016,11,2015,Yale University,11,2015,1
358,Yeshiva University,164,2016,201-300,2015,Yeshiva University,171,2015,136
359,Yonsei University,301-350,2016,201-300,2015,Yonsei University,98,2015,252
360,York University,301-350,2016,401-500,2014,York University,337,2015,163


Controlliamo che la Aarhus University abbia effettivamente il valore `49` come differenza:

In [55]:
times_shanghai_cwur[times_shanghai_cwur['university_name']=='Aarhus University']

,university_name,world_rank_times,year_times,world_rank_shanghai,year_shanghai,institution,world_rank,year,differenza_max_min
2,Aarhus University,106,2016,73,2015,Aarhus University,122,2015,49


### 5. Consider only the most recent data point of the `times` dataset. Compute the number of male and female students for each country.¶

Vengono quindi considerati i *data points* più recenti del dataset `times`.

In [56]:
times_max_2 = times[times['year'] == times['year'].max()]
times_max_2.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
1803,1,California Institute of Technology,United States of America,95.6,64.0,97.6,99.8,97.8,95.2,"2,243",6.9,27%,33 : 67,2016
1804,2,University of Oxford,United Kingdom,86.5,94.4,98.9,98.8,73.1,94.2,"19,919",11.6,34%,46 : 54,2016


Viene richiesto che i valori nulli non vengano considerati per i successivi passaggi per la colonna `female_male_ratio` e nemmeno i valori contenenti il `-`: i valori di questo tipo vengono sono visti come mancanti e non utilizzati per le operazioni ancora da svolgere. 

In [57]:
times_max_2 = times_max_2[times_max_2['female_male_ratio'].notnull()]
times_max_2 = times_max_2[~times_max_2['female_male_ratio'].str.contains('-')]

Viene verificata il tipo della colonna `num_students` in quanto ambigua.

In [58]:
times_max_2['num_students']

1803     2,243
1804    19,919
1805    15,596
1806    18,812
1807    11,074
         ...  
2597    31,618
2598    21,958
2599    31,268
2601    10,117
2602     8,663
Name: num_students, Length: 736, dtype: object

Non essendo un effettivo *float number* si crea una funzione che rimpiazzi la `,` col `.` e restituisca un valore intero.

In [59]:
def convert_to_int(col):
    repl = int(col.replace(',', ''))
    
    return repl

La colonna `num_students` viene convertita tramite la funzione `convert_to_int` in `int number`, passaggio necessario per poter svolgere le successive operazioni matematiche.

In [60]:
times_max_2['num_students'] = times_max_2['num_students'].apply(convert_to_int)
times_max_2.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
1803,1,California Institute of Technology,United States of America,95.6,64.0,97.6,99.8,97.8,95.2,2243,6.9,27%,33 : 67,2016
1804,2,University of Oxford,United Kingdom,86.5,94.4,98.9,98.8,73.1,94.2,19919,11.6,34%,46 : 54,2016
1805,3,Stanford University,United States of America,92.5,76.3,96.2,99.9,63.3,93.9,15596,7.8,22%,42 : 58,2016
1806,4,University of Cambridge,United Kingdom,88.2,91.5,96.7,97.0,55.0,92.8,18812,11.8,34%,46 : 54,2016
1807,5,Massachusetts Institute of Technology,United States of America,89.4,84.0,88.6,99.7,95.4,92.0,11074,9.0,33%,37 : 63,2016


Si creano due funzioni (una per il campo `femmine` e una per il campo `maschi`) che ricerchino esattamente quanto richiesto tramite il match della regular expression.

In [61]:
def female_assoluto(col):
    # Si trova il match con il group regex
    fem_regex = re.match('^(\d+) : (\d+)$', col['female_male_ratio'])
        
    # Si convertono in interi i valori
    female = round(int(fem_regex.group(1)) * col['num_students']/100)
    
    return female
    
def male_assoluto(col):
    # Si trova il match con il group regex
    male_regex = re.match('^(\d+) : (\d+)$', col['female_male_ratio'])
    
    # Si convertono in interi i valori
    male = round(int(male_regex.group(2)) * col['num_students']/100)
    
    return male

Si applicano le due funzioni al dataframe `times_max_2` creando due colonne che corrispondano ai valori assoluti di `female` e di `male`:

In [62]:
times_max_2['female'] = times_max_2.apply(female_assoluto, axis=1)
times_max_2['male'] = times_max_2.apply(male_assoluto, axis=1)

Infine, si richiama il dataframe con le nuove colonne per essere certi della correttezza del risultato richiesto, scegliendo le colonne per una migliore visualizzazione:

In [63]:
times_max_2[['country', 'year', 'female_male_ratio', 'female', 'male']]

,country,year,female_male_ratio,female,male
1803,United States of America,2016,33 : 67,740,1503
1804,United Kingdom,2016,46 : 54,9163,10756
1805,United States of America,2016,42 : 58,6550,9046
1806,United Kingdom,2016,46 : 54,8654,10158
1807,United States of America,2016,37 : 63,4097,6977
...,...,...,...,...,...
2597,China,2016,29 : 71,9169,22449
2598,South Korea,2016,48 : 52,10540,11418
2599,Turkey,2016,36 : 64,11256,20012
2601,Japan,2016,28 : 72,2833,7284


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

Viene creata una funzione che computi il rapporto fra `femmine` e `maschi`. Nel caso in cui i `maschi`, che si trovano al denominatore, fossero pari a `0`, verrà restituito il valore `100` in quanto al numeratore si troveranno le `femmine` uguali a `100`. Così facendo il rapporto sarà pari a `100` indicando una università totalmente femminile.

In [64]:
def fem_mal(col):
    # Ricerca il match nella colonna female_male_ratio
    find_male_zero = re.match('^(?P<female>\d+) : (?P<male>\d+)$', col['female_male_ratio'])
    
    # converto in interi i valori di female e male (da stringhe)
    female = int(find_male_zero.group('female'))
    male = int(find_male_zero.group('male'))
    
    # se 'male' fosse uguale a 0, ritorna direttamente il valore 100, considerando l'università
    # completamente femminile
    if male == 0:
        return 100
    # Altrimenti ritorna il rapporto
    else:
        return female/male

Applica la funzione `f_m_ratio_decimal`:

In [65]:
times_max_2['f_m_ratio_decimal'] = times_max_2.apply(fem_mal, axis=1)

Se ne controlla la correttezza:

In [66]:
times_max_2.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male,f_m_ratio_decimal
1803,1,California Institute of Technology,United States of America,95.6,64.0,97.6,99.8,97.8,95.2,2243,6.9,27%,33 : 67,2016,740,1503,0.492537
1804,2,University of Oxford,United Kingdom,86.5,94.4,98.9,98.8,73.1,94.2,19919,11.6,34%,46 : 54,2016,9163,10756,0.851852
1805,3,Stanford University,United States of America,92.5,76.3,96.2,99.9,63.3,93.9,15596,7.8,22%,42 : 58,2016,6550,9046,0.724138
1806,4,University of Cambridge,United Kingdom,88.2,91.5,96.7,97.0,55.0,92.8,18812,11.8,34%,46 : 54,2016,8654,10158,0.851852
1807,5,Massachusetts Institute of Technology,United States of America,89.4,84.0,88.6,99.7,95.4,92.0,11074,9.0,33%,37 : 63,2016,4097,6977,0.587302


Viene applicata la funzione `mean()` per calcolare la media di colonna:

In [67]:
times_max_2['f_m_ratio_decimal'].mean()

1.2169695629288875

Troviamo quindi quelle università che presentano un rapporto `female_male` al di sotto della media:

In [68]:
sotto_media = times_max_2[times_max_2['f_m_ratio_decimal'] < times_max_2['f_m_ratio_decimal'].mean()]
sotto_media

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male,f_m_ratio_decimal
1803,1,California Institute of Technology,United States of America,95.6,64.0,97.6,99.8,97.8,95.2,2243,6.9,27%,33 : 67,2016,740,1503,0.492537
1804,2,University of Oxford,United Kingdom,86.5,94.4,98.9,98.8,73.1,94.2,19919,11.6,34%,46 : 54,2016,9163,10756,0.851852
1805,3,Stanford University,United States of America,92.5,76.3,96.2,99.9,63.3,93.9,15596,7.8,22%,42 : 58,2016,6550,9046,0.724138
1806,4,University of Cambridge,United Kingdom,88.2,91.5,96.7,97.0,55.0,92.8,18812,11.8,34%,46 : 54,2016,8654,10158,0.851852
1807,5,Massachusetts Institute of Technology,United States of America,89.4,84.0,88.6,99.7,95.4,92.0,11074,9.0,33%,37 : 63,2016,4097,6977,0.587302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,601-800,Xidian University,China,17.9,12.8,12.1,8.9,83.7,-,31618,16.4,2%,29 : 71,2016,9169,22449,0.408451
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,21958,15.3,3%,48 : 52,2016,10540,11418,0.923077
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,31268,28.7,2%,36 : 64,2016,11256,20012,0.562500
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,10117,12.1,8%,28 : 72,2016,2833,7284,0.388889


### 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

Considerando il dataframe poco sopra creato (sulle università che presentavano un rapporto femmine/maschi sotto la media), si raggruppa per `country` e, sul numero di studenti, si sommano per ottenere il totale degli studenti per paese.

In [99]:
stud_paese_sotto_media_parz = sotto_media.groupby("country", as_index=False)["num_students"].agg(sum)
stud_paese_sotto_media_parz.head()

,country,num_students
0,Australia,295021
1,Austria,79242


Si crea un nuovo df con il numero di studenti totali:

In [100]:
stud_paese_totali = times_max_2.groupby("country", as_index=False)["num_students"].agg(sum)
stud_paese_totali.head()

,country,num_students
0,Argentina,108373
1,Australia,713376


Si fondono i due dataframe:

In [101]:
stud_parz_tot_merge = pd.merge(stud_paese_sotto_media_parz, stud_paese_totali, on='country', suffixes=('_parz', '_tot'))
stud_parz_tot_merge.head()

,country,num_students_parz,num_students_tot
0,Australia,295021,713376
1,Austria,79242,134477
2,Bangladesh,62716,62716
3,Belgium,116129,169661
4,Brazil,438476,494251


Si crea la colonna desiderata del rapporto:

In [102]:
stud_parz_tot_merge['ratio_parz_tot'] = (stud_parz_tot_merge['num_students_parz']/stud_parz_tot_merge['num_students_tot'])
stud_parz_tot_merge.head()

,country,num_students_parz,num_students_tot,ratio_parz_tot
0,Australia,295021,713376,0.413556
1,Austria,79242,134477,0.589261
2,Bangladesh,62716,62716,1.000000
3,Belgium,116129,169661,0.684477
4,Brazil,438476,494251,0.887152


### 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

Viene letto il dataframe:

In [103]:
educational_attainment = pd.read_csv('dataset_progetto/educational_attainment_supplementary_data.csv')
educational_attainment.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN


Si andranno ad eliminare quei valori nulli nelle colonne presenti nel parametro `subset` di `dropna()`:

In [74]:
educational_attainment.dropna(subset=['country_name', 'series_name'], inplace=True)

Si controlla se quanto svolto è corretto:

In [75]:
educational_attainment[educational_attainment['country_name'].isnull()]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


In [76]:
educational_attainment[educational_attainment['series_name'].isnull()]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


### 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

Si controlla quali sono le colonne attualmente presenti in `educational_attainment`:

In [77]:
educational_attainment.columns

Index(['country_name', 'series_name', '1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015'],
      dtype='object')

Il risultato finale che si vuole ottenere prevede di trasformare il dataframe nel *wide format* in un dataframe in un formato *long*, ovvero convertire le colonne per renderle come valori di riga. Sarà necessario anche modificare il nome di colonna in `year`.
Per poter svolgere tale compito, si utilizza la funzione `.melt()` adibita a tale scopo:

In [78]:
melt_educational_df = pd.melt(educational_attainment, id_vars=['country_name', 'series_name'], var_name="year")
melt_educational_df.head()

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

Si raggruppa per `university_name` sul dataframe iniziale `times` ottenendo il `count` del numero di volte in cui la singola università compare nel ranking.

In [79]:
times_group_rank = times.groupby('university_name', as_index=False).count()[['university_name', 'world_rank']]

La stessa operazione dovrà essere svolta anche sui restanti dataset:

In [80]:
shanghai_group_rank = shanghai_db.groupby('university_name', as_index=False).count()[['university_name', 'world_rank']]

In [81]:
cwur_group_rank = cwur_db.groupby('institution', as_index=False).count()[['institution', 'world_rank']]

A questo punto sarà necessario fondere i tre dataset:

In [82]:
t_s_fusion_intermezzo = pd.merge(times_group_rank, shanghai_group_rank, on='university_name', suffixes=('_times', '_shanghai'), how='outer')
t_s_fusion_intermezzo.head(2)

,university_name,world_rank_times,world_rank_shanghai
0,AGH University of Science and Technology,1.0,NaN
1,Aalborg University,5.0,2.0


In [83]:
t_s_c_fusion_finale = pd.merge(t_s_fusion_intermezzo, cwur_group_rank, left_on='university_name', right_on='institution', how='outer')
t_s_c_fusion_finale.head()

,university_name,world_rank_times,world_rank_shanghai,institution,world_rank
0,AGH University of Science and Technology,1.0,NaN,AGH University of Science and Technology,2.0
1,Aalborg University,5.0,2.0,Aalborg University,2.0
2,Aalto University,5.0,7.0,Aalto University,2.0
3,Aarhus University,6.0,11.0,Aarhus University,2.0
4,Aberystwyth University,5.0,NaN,Aberystwyth University,2.0


Avendo a disposizione 3 ranking, si conterà quante volte una specifica università viene a trovarsi in ogni rank ragionando sui presenza o meno dei valori nulli:

In [84]:
def conta_rank(col):
    cont_tot = 3
    if pd.isnull(col['world_rank_times']):
        cont_tot -= 1
    elif pd.isnull(col['world_rank_shanghai'] ):
        cont_tot -= 1
    elif pd.isnull(col['world_rank'] ):
        cont_tot -= 1
        
    return cont_tot

Viene applicata la funzione `conta_rank` sulle colonne:

In [85]:
t_s_c_fusion_finale['conteggio_rank'] = t_s_c_fusion_finale.apply(conta_rank, axis=1)

Si visualizzano solo le colonne di interesse:

In [86]:
t_s_c_fusion_finale[['university_name', 'institution', 'conteggio_rank']]

,university_name,institution,conteggio_rank
0,AGH University of Science and Technology,AGH University of Science and Technology,2
1,Aalborg University,Aalborg University,3
2,Aalto University,Aalto University,3
3,Aarhus University,Aarhus University,3
4,Aberystwyth University,Aberystwyth University,2
...,...,...,...
1447,NaN,École Polytechnique de Montréal,2
1448,NaN,École centrale de Lyon,2
1449,NaN,École normale supérieure - Paris,2
1450,NaN,École normale supérieure de Cachan,2


### 11. In the `times` ranking, compute the number of times each university appears

In [87]:
times.groupby('university_name', as_index=False)['world_rank'].count()

,university_name,world_rank
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


### 12. Find the universities that appear at most twice in the times ranking.

In [88]:
times_ranking_df = times.groupby('university_name').size()

In [89]:
times_ranking_df[times_ranking_df < 3]

university_name
AGH University of Science and Technology    1
Adam Mickiewicz University                  1
Aix-Marseille University                    1
Ajou University                             1
Alexandru Ioan Cuza University              1
                                           ..
Yokohama City University                    1
Yokohama National University                1
Yuan Ze University                          2
Yıldız Technical University                 1
Örebro University                           1
Length: 408, dtype: int64

### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

Si procede con un `pd.merge` dei diversi dataframe:

In [90]:
t_s_fusion_db = pd.merge(times, shanghai_db, on='university_name', suffixes=('_times', '_shanghai'))

In [91]:
t_s_c_fusion_db = pd.merge(t_s_fusion_db, cwur_db, left_on='university_name', right_on='institution')

In [92]:
t_s_c_fusion_db.columns

Index(['world_rank_times', 'university_name', 'country_x', 'teaching',
       'international', 'research', 'citations_x', 'income',
       'total_score_times', 'num_students', 'student_staff_ratio',
       'international_students', 'female_male_ratio', 'year_times',
       'world_rank_shanghai', 'national_rank_x', 'total_score_shanghai',
       'alumni', 'award', 'hici', 'ns', 'pub', 'pcp', 'year_shanghai',
       'world_rank', 'institution', 'country_y', 'national_rank_y',
       'quality_of_education', 'alumni_employment', 'quality_of_faculty',
       'publications', 'influence', 'citations_y', 'broad_impact', 'patents',
       'score', 'year'],
      dtype='object')

In [93]:
t_s_c_fusion_db.head(2)

,world_rank_times,university_name,country_x,teaching,international,research,citations_x,income,total_score_times,num_students,...,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations_y,broad_impact,patents,score,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",...,7,9,1,1,1,1,NaN,5,100.0,2012
1,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",...,1,1,1,1,1,1,NaN,7,100.0,2013


Si scelgono solamente le colonne di interesse:

In [94]:
t_s_c_fusion_db = t_s_c_fusion_db[['university_name', 'institution', 'world_rank_times',  'year_times', 'world_rank_shanghai', 'year_shanghai', 'world_rank', 'year']]

In [95]:
t_s_c_fusion_db

,university_name,institution,world_rank_times,year_times,world_rank_shanghai,year_shanghai,world_rank,year
0,Harvard University,Harvard University,1,2011,1,2005,1,2012
1,Harvard University,Harvard University,1,2011,1,2005,1,2013
2,Harvard University,Harvard University,1,2011,1,2005,1,2014
3,Harvard University,Harvard University,1,2011,1,2005,1,2015
4,Harvard University,Harvard University,1,2011,1,2006,1,2012
...,...,...,...,...,...,...,...,...
33173,University of Tehran,University of Tehran,601-800,2016,301-400,2014,768,2015
33174,University of Tehran,University of Tehran,601-800,2016,201-300,2015,749,2014
33175,University of Tehran,University of Tehran,601-800,2016,201-300,2015,768,2015
33176,Tokyo University of Science,Tokyo University of Science,601-800,2016,401-500,2014,413,2014


E' necessario convertire la colonna `world_rank` in stringa in quanto si tratta di un valore numerico, mentre le altre colonne di world rank sono considerate come oggetti:

In [96]:
t_s_c_fusion_db['world_rank'] = t_s_c_fusion_db['world_rank'].apply(str)

Viene richiesto di ottenere soltanto quelle istanze che si presentano nello stesso rank e, successivamente, nello stesso anno.

Nello stesso rank:

In [97]:
t_s_c_fusion_db_same_rank = t_s_c_fusion_db[(t_s_c_fusion_db['world_rank_times'] == t_s_c_fusion_db['world_rank_shanghai']) & (t_s_c_fusion_db['world_rank_times'] == t_s_c_fusion_db['world_rank'])]
t_s_c_fusion_db_same_rank.head()

,university_name,institution,world_rank_times,year_times,world_rank_shanghai,year_shanghai,world_rank,year
0,Harvard University,Harvard University,1,2011,1,2005,1,2012
1,Harvard University,Harvard University,1,2011,1,2005,1,2013
2,Harvard University,Harvard University,1,2011,1,2005,1,2014
3,Harvard University,Harvard University,1,2011,1,2005,1,2015
4,Harvard University,Harvard University,1,2011,1,2006,1,2012


Nello stesso anno:

In [98]:
t_s_c_fusion_db_same_year = t_s_c_fusion_db_same_rank[(t_s_c_fusion_db_same_rank['year_times'] == t_s_c_fusion_db_same_rank['year_shanghai']) & (t_s_c_fusion_db_same_rank['year_times'] == t_s_c_fusion_db_same_rank['year'])]
t_s_c_fusion_db_same_year.head()

,university_name,institution,world_rank_times,year_times,world_rank_shanghai,year_shanghai,world_rank,year
649,Stanford University,Stanford University,2,2013,2,2013,2,2013


Ottenendo come risultato conclusivo che la *Standford University* è l'unica università che viene a trovarsi in tutti e tre i ranking, nello stesso anno, nella stessa identica posizione.